# Project Sidewalk - get crossing features

In [ ]:
# Standard library and path imports
import set_path
import json
import requests

# Third-party library imports
import geopandas as gpd
import folium

# Local or project-specific imports
import plot_utils
import settings as st

if st.my_run == "azure":
    import config_azure as cf
elif st.my_run == "local":
    import config as cf

In [ ]:
if st.my_run == "azure":
    os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

## 1. Get project sidewalk data

### Get the correct url for a small test neighborhood or the whole city

In [ ]:
base_url = "https://sidewalk-amsterdam.cs.washington.edu/v2/access/attributesWithLabels?lat1={}&lng1={}&lat2={}&lng2={}" 
amsterdam_bbox_coords = (52.265, 4.76, 52.435, 5.07)
url = base_url.format(*amsterdam_bbox_coords)

### Try to read the data from local dump if available else go to the Project Sidewalk API

In [ ]:
local_dump = cf.out_folder + url.replace('/', '|')

try:
    project_sidewalk_labels = json.load(open(local_dump, 'r'))
except Exception:
    print("Couldn't load local dump")
    project_sidewalk_labels = requests.get(url.format(*amsterdam_bbox_coords)).json()
    json.dump(project_sidewalk_labels, open(local_dump, 'w'))

### Create sidewalk data geodataframe

In [ ]:
p_sw_gdf = gpd.GeoDataFrame.from_features(project_sidewalk_labels['features'], crs=st.CRS_map)
p_sw_gdf = p_sw_gdf.to_crs(st.CRS)

### Get mask of area

In [ ]:
# Import areas
df_areas = gpd.read_file(cf.output_pilot_area)
polygon = df_areas.to_crs(crs=st.CRS).unary_union

### Extract sidewalk data in polygon of area

In [ ]:
sw_data_in_area_mask = p_sw_gdf['geometry'].apply(lambda geom: polygon.contains(geom))
p_sw_area_gdf = p_sw_gdf.loc[sw_data_in_area_mask]

### Inspect data

In [ ]:
if p_sw_area_gdf.empty:
    print('No sidewalk data available in area')

p_sw_area_gdf['label_tags'] = str(p_sw_area_gdf['label_tags'])
p_sw_area_gdf

In [ ]:
# set True for satellite background, False for regular background
satellite = False

# Create Folium map
map = folium.Map(
    location=[52.350547922223434, 4.794019242371844], tiles=plot_utils.generate_map_params(satellite=satellite),
    min_zoom=10, max_zoom=25, zoom_start=17,
    zoom_control=True, control_scale=True, control=False
    )

# Add traffic sign features
feature_names = p_sw_area_gdf.columns.tolist()
feature_names.remove('geometry')
tooltip = plot_utils.gen_tooltip(feature_names, feature_names)
geo_j = folium.GeoJson(p_sw_area_gdf, tooltip=tooltip, style_function=lambda x: {"color": "orange", "weight": 10}).add_to(map)

map

### Save project sidewalk data in selected area as csv

In [ ]:
# Save project sidewalk crossing features as geopackage
p_sw_area_gdf.to_file(cf.output_project_sidewalk_crossing_features, driver='GPKG')